<a href="https://colab.research.google.com/github/jdblackstar/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-regression-2/212A_Regression_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

#Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [7]:
# first, let's look at the data

print(df.dtypes)
df.head()

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# noticing that 'created' is an object, and it should be a datetime for us to
# know the sequential order of dates

df['created'] = pd.to_datetime(df['created'])

# only looking for month, so let's ditch everything else
df['created'] = df['created'].dt.month

In [10]:
df.dtypes.head()

bathrooms          float64
bedrooms             int64
created              int64
description         object
display_address     object
dtype: object

In [11]:
# didn't realize that would turn it to an int, but it should be fine
df['created'].value_counts()

6    16973
4    16217
5    15627
Name: created, dtype: int64

In [14]:
# when we make train and test dataframes, they should be 31,844 and 16,973
# rows respectively

train = df[df['created'] < 6]
test = df[df['created'] == 6]

target = 'price'
y_train = train[target]
y_test = test[target]

guess = y_train.mean()
guess

3575.604007034292

In [18]:
from sklearn.metrics import mean_absolute_error

# train error

y_train_pred = [guess] * len(y_train)
train_mae = mean_absolute_error(y_train, y_train_pred)
print("Training data error:", train_mae, '%')

Training data error: 1201.8811133682555 %


In [20]:
# test error

y_test_pred = [guess] * len(y_test)
test_mae = mean_absolute_error(y_test, y_test_pred)
print("Test data error:", test_mae, '%')

Test data error: 1197.7088871089013 %


In [34]:
# with 1 feature - cats allowed
import plotly.express as px
from sklearn.linear_model import LinearRegression
# plotly and regression imports as well
# and instantiate the class
model = LinearRegression()

Linear Regression, dependent on: ['cats_allowed']


In [46]:
features = ['cats_allowed']
X_train = train[features]
X_test = test[features]
print('Linear Regression, dependent on:', features)

Linear Regression, dependent on: {features}


In [47]:
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print('Train Error:', mae, '%')

Train Error: 1198.2932176523789 %


In [36]:
# now with dogs

features = ['dogs_allowed']
X_train = train[features]
X_test = test[features]
print('Linear Regression, dependent on:', features)

Linear Regression, dependent on: ['dogs_allowed']


In [37]:
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print('Train Error:', mae, '%')

Train Error: 1197.3790891409164 %


In [38]:
# multi-feature (cats and dogs)

features = ['cats_allowed', 'dogs_allowed']
print('Linear Regression, dependent on:', features)

X_train = train[features]
X_test = test[features]

Linear Regression, dependent on: ['cats_allowed', 'dogs_allowed']


In [39]:
# fit model
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# apply model to new 2d train

y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print('Train Error: ', mae, '%')

In [42]:
# apply model to new 2d test

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('Test Error: ', mae, '%')

Test Error: 1194.17 percentage points


In [43]:
# little more human readable as a coefficient. i believe this is saying that if
# cats are allowed, the price will decrease by ~$138, wheres if dogs are allowed,
# it will increase by ~$345

model.coef_

array([-138.11834996,  345.83111135])

In [0]:
# going to continue some of our fun we had yesterday and try to see if there is
# correlation between price of an apartment and proximity to Time's Square
# going to attempt once with basic python, and once with a library